In [3]:
import datetime
import time
import pandas
import requests
import json
import logging
import pytz
import pyarrow
from google.oauth2 import service_account
from google.cloud import bigquery

In [42]:
# key credentials and access point
# Documentation link 👇🏼 used as a guide
# https://github.com/googleapis/python-bigquery/blob/35627d145a41d57768f19d4392ef235928e00f72/samples/load_table_dataframe.py
key_path= "/Users/ezekielmauricio/Documents/code/data/projectExample/subdir/credentials.json"
project_id="metro-rail-etl-v1"
dataset_id = "metro_data"
table="metro_data_set"
table_id="{}.{}.{}".format(project_id, dataset_id, table)
print("********* NAME OF TABLE IS", table_id)

def load_table_dataframe(key_path,project_id,table_id):
    credentials = service_account.Credentials.from_service_account_file(
        key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
        )
    # Construct a BigQuery client object.
    client = bigquery.Client(credentials=credentials, project=project_id)
    # retrieve data from API call
    url = 'https://data.texas.gov/resource/u57e-u5kq.json?year_num=2022'
    response = requests.get(url)
    statusResponse = requests.request("GET", url)
    print(statusResponse)
    res_data = statusResponse.json()
    time.sleep(2) # give data time to load
    # Convert data to proper data types
    for row in res_data:
        row["route_sort_order"] = int(row["route_sort_order"])
        row["ridership_average"] = float(row["ridership_average"])
        row["ytd_total_ridership_count"] = float(row["ytd_total_ridership_count"])
        row["qtd_total_ridership_count"] = float(row["qtd_total_ridership_count"])
        row["py_qtd_total_ridership_count"] = float(row["py_qtd_total_ridership_count"])
        row["py_ytd_total_ridership_count"] = float(row["py_ytd_total_ridership_count"])
        row["py_ridership_average"] = float(row["py_ridership_average"])
        row["year_num"] = int(row["year_num"])
        row["month_end_date"] = pandas.to_datetime(row["month_end_date"], format="%Y%m%d").date()
        row["fiscal_year_number"] = int(row["fiscal_year_number"])
        row["fiscal_month_number"] = int(row["fiscal_month_number"])
        row["fiscal_quarter_number"] = int(row["fiscal_quarter_number"])
        row["show_on_dashboard"] = int(row["show_on_dashboard"])

    # create a new dataframe
    dataframe = pandas.DataFrame(
    res_data,
    # In the loaded table, the column order reflects the order of the
    # columns in the DataFrame.
        columns=[
            "route_sort_order",
            "ridership_average",
            "ytd_total_ridership_count",
            "qtd_total_ridership_count",
            "py_qtd_total_ridership_count",
            "py_ytd_total_ridership_count",
            "py_ridership_average",
            "year_num",
            "month_name",
            "month_short_name",
            "month_end_date",
            "month_year",
            "day_type",
            "mode_name",
            "sub_mode",
            "route_name",
            "data_as_of",
            "show_on_dashboard",
            "fiscal_year_number",
            "fiscal_month_number",
            "fiscal_quarter_number"
        ]
    )
    print(dataframe)
    job_config = bigquery.LoadJobConfig(
        schema = [
            bigquery.SchemaField("route_sort_order", bigquery.enums.SqlTypeNames.INTEGER),    
            bigquery.SchemaField("ridership_average", bigquery.enums.SqlTypeNames.FLOAT),    
            bigquery.SchemaField("ytd_total_ridership_count", bigquery.enums.SqlTypeNames.FLOAT),    
            bigquery.SchemaField("qtd_total_ridership_count", bigquery.enums.SqlTypeNames.FLOAT),    
            bigquery.SchemaField("py_qtd_total_ridership_count", bigquery.enums.SqlTypeNames.FLOAT),    
            bigquery.SchemaField("py_ytd_total_ridership_count", bigquery.enums.SqlTypeNames.FLOAT),    
            bigquery.SchemaField("py_ridership_average", bigquery.enums.SqlTypeNames.FLOAT),    
            bigquery.SchemaField("year_num", bigquery.enums.SqlTypeNames.INTEGER),    
            bigquery.SchemaField("month_name", bigquery.enums.SqlTypeNames.STRING),    
            bigquery.SchemaField("month_short_name", bigquery.enums.SqlTypeNames.STRING),    
            bigquery.SchemaField("month_end_date", bigquery.enums.SqlTypeNames.DATE),    
            bigquery.SchemaField("month_year", bigquery.enums.SqlTypeNames.STRING),    
            bigquery.SchemaField("day_type", bigquery.enums.SqlTypeNames.STRING),   
            bigquery.SchemaField("mode_name", bigquery.enums.SqlTypeNames.STRING),    
            bigquery.SchemaField("sub_mode", bigquery.enums.SqlTypeNames.STRING),    
            bigquery.SchemaField("route_name", bigquery.enums.SqlTypeNames.STRING),    
            bigquery.SchemaField("data_as_of", bigquery.enums.SqlTypeNames.STRING),    
            bigquery.SchemaField("show_on_dashboard", bigquery.enums.SqlTypeNames.INTEGER),    
            bigquery.SchemaField("fiscal_year_number", bigquery.enums.SqlTypeNames.INTEGER),    
            bigquery.SchemaField("fiscal_month_number", bigquery.enums.SqlTypeNames.INTEGER),    
            bigquery.SchemaField("fiscal_quarter_number", bigquery.enums.SqlTypeNames.INTEGER)
        ],
        # Optionally, set the write disposition. BigQuery appends loaded rows
        # to an existing table by default, but with WRITE_TRUNCATE write
        # disposition it replaces the table with the loaded data.
        write_disposition="WRITE_TRUNCATE"
    )

    job = client.load_table_from_dataframe(
        dataframe, table_id, job_config=job_config
    )  # Make an API request.
    job.result()  # Wait for the job to complete.

    data = client.get_table(table_id)  
    return data

data = load_table_dataframe(key_path,project_id, table_id)

********* NAME OF TABLE IS metro-rail-etl-v1.metro_data.metro_data_set
<Response [200]>
     route_sort_order  ridership_average  ytd_total_ridership_count  \
0                 445             0.0000                    0.00000   
1                 982             0.0000                    0.00000   
2                 315          1873.0000                11855.59993   
3                 490             0.0000                    0.00000   
4                 201           702.0000                 5585.99992   
..                ...                ...                        ...   
995               999          3021.0000                16408.99989   
996               481           645.2000                 6219.19982   
997                19          1524.7739                 1524.77390   
998               981             0.0000                    0.00000   
999               103             0.0000                    0.00000   

     qtd_total_ridership_count  py_qtd_total_ridership_coun

,route_sort_order,ridership_average,ytd_total_ridership_count,qtd_total_ridership_count,py_qtd_total_ridership_count,py_ytd_total_ridership_count,py_ridership_average,year_num,month_name,month_short_name,...,day_type,mode_name,mode_name,sub_mode,route_name,data_as_of,show_on_dashboard,fiscal_year_number,fiscal_month_number,fiscal_quarter_number
0,445,0,0,0,0,0,0,2022,September,09-Sep,...,Weekday,MetroBus,MetroBus,Special Service Routes,445-New Life Shuttle,10/31/2022,1,2022,12,4
1,982,0,0,0,0,0,0,2022,January,01-Jan,...,Saturday,MetroExpress,MetroExpress,MetroExpress,982-Pavilion Express,10/31/2022,1,2022,4,2
2,315,1873,11855.59993,5789.09995,5642.84364,11662.59359,1753.66664,2022,March,03-Mar,...,Saturday,MetroBus,MetroBus,Crosstown Routes,315-Ben White,10/31/2022,1,2022,6,2
3,490,0,0,0,0,0,0,2022,February,02-Feb,...,Sunday,MetroBus,MetroBus,Special Service Routes,490-HEB Shuttle,10/31/2022,1,2022,5,2
4,201,702,5585.99992,2452.69998,1202.4666,4766.86654,674.66664,2022,March,03-Mar,...,Sunday,MetroBus,MetroBus,Feeder Routes,201-Southpark Meadows,10/31/2022,1,2022,6,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,999,3021,16408.99989,6564.99995,5794.99998,14435.99993,2484,2022,February,02-Feb,...,Saturday,MetroAccess,MetroAccess,MetroAccess,MA-MetroAccess,10/31/2022,1,2022,5,2
996,481,645.2,6219.19982,2119.29991,0,0,0,2022,June,06-Jun,...,Saturday,MetroBus,MetroBus,Special Service Routes,481-Night Owl North Lamar,10/31/2022,1,2022,9,3
997,19,1524.7739,1524.7739,1524.7739,1372.59995,1372.59995,1372.59995,2022,October,10-Oct,...,Sunday,MetroBus,MetroBus,Local Routes,19-Bull Creek,10/31/2022,1,2023,1,1
998,981,0,0,0,0,0,0,2022,April,04-Apr,...,Weekday,MetroExpress,MetroExpress,MetroExpress,981-Oak Knoll Express,10/31/2022,1,2022,7,3


In [33]:
# exploratory
url = "https://data.texas.gov/resource/u57e-u5kq.json?year_num=2022"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
else:
    print("Failed to retrieve data. Status code: {}".format(response.status_code))




for row in data:
    row["route_sort_order"] = int(row["route_sort_order"])
    row["ridership_average"] = float(row["ridership_average"])
    row["ytd_total_ridership_count"] = float(row["ytd_total_ridership_count"])
    row["qtd_total_ridership_count"] = float(row["qtd_total_ridership_count"])
    row["py_qtd_total_ridership_count"] = float(row["py_qtd_total_ridership_count"])
    row["py_ytd_total_ridership_count"] = float(row["py_ytd_total_ridership_count"])
    row["py_ridership_average"] = float(row["py_ridership_average"])
    row["year_num"] = int(row["year_num"])
    row["month_end_date"] = int(row["month_end_date"])
    row["fiscal_year_number"] = int(row["fiscal_year_number"])
    row["fiscal_month_number"] = int(row["fiscal_month_number"])
    row["fiscal_quarter_number"] = int(row["fiscal_quarter_number"])
    row["month_end_date"] = pandas.to_datetime(row["month_end_date"], format="%Y%m%d").date()

    print (row["month_end_date"])

df = pandas.DataFrame(data)




2022-09-30
2022-01-31
2022-03-31
2022-02-28
2022-03-31
2022-06-30
2022-02-28
2022-09-30
2022-03-31
2022-10-31
2022-10-31
2022-07-31
2022-02-28
2022-10-31
2022-10-31
2022-03-31
2022-09-30
2022-05-31
2022-04-30
2022-09-30
2022-05-31
2022-10-31
2022-06-30
2022-04-30
2022-07-31
2022-02-28
2022-09-30
2022-02-28
2022-10-31
2022-04-30
2022-06-30
2022-10-31
2022-09-30
2022-09-30
2022-05-31
2022-05-31
2022-07-31
2022-03-31
2022-02-28
2022-06-30
2022-06-30
2022-02-28
2022-09-30
2022-01-31
2022-08-31
2022-08-31
2022-10-31
2022-10-31
2022-02-28
2022-01-31
2022-08-31
2022-04-30
2022-04-30
2022-01-31
2022-08-31
2022-10-31
2022-09-30
2022-09-30
2022-07-31
2022-10-31
2022-06-30
2022-10-31
2022-04-30
2022-02-28
2022-06-30
2022-02-28
2022-08-31
2022-05-31
2022-10-31
2022-07-31
2022-07-31
2022-08-31
2022-03-31
2022-04-30
2022-06-30
2022-06-30
2022-05-31
2022-04-30
2022-02-28
2022-01-31
2022-02-28
2022-06-30
2022-07-31
2022-01-31
2022-05-31
2022-04-30
2022-05-31
2022-09-30
2022-01-31
2022-02-28
2022-01-31

In [12]:
df

,route_sort_order,ridership_average,ytd_total_ridership_count,qtd_total_ridership_count,py_qtd_total_ridership_count,py_ytd_total_ridership_count,py_ridership_average,year_num,month_name,month_short_name,...,month_year,day_type,mode_name,sub_mode,route_name,data_as_of,show_on_dashboard,fiscal_year_number,fiscal_month_number,fiscal_quarter_number
0,445,0.0000,0.00000,0.00000,0.00000,0.00000,0.00000,2022,September,09-Sep,...,Sep-20,Weekday,MetroBus,Special Service Routes,445-New Life Shuttle,10/31/2022,1,2022,12,4
1,982,0.0000,0.00000,0.00000,0.00000,0.00000,0.00000,2022,January,01-Jan,...,Jan-20,Saturday,MetroExpress,MetroExpress,982-Pavilion Express,10/31/2022,1,2022,4,2
2,315,1873.0000,11855.59993,5789.09995,5642.84364,11662.59359,1753.66664,2022,March,03-Mar,...,Mar-20,Saturday,MetroBus,Crosstown Routes,315-Ben White,10/31/2022,1,2022,6,2
3,490,0.0000,0.00000,0.00000,0.00000,0.00000,0.00000,2022,February,02-Feb,...,Feb-20,Sunday,MetroBus,Special Service Routes,490-HEB Shuttle,10/31/2022,1,2022,5,2
4,201,702.0000,5585.99992,2452.69998,1202.46660,4766.86654,674.66664,2022,March,03-Mar,...,Mar-20,Sunday,MetroBus,Feeder Routes,201-Southpark Meadows,10/31/2022,1,2022,6,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,999,3021.0000,16408.99989,6564.99995,5794.99998,14435.99993,2484.00000,2022,February,02-Feb,...,Feb-20,Saturday,MetroAccess,MetroAccess,MA-MetroAccess,10/31/2022,1,2022,5,2
996,481,645.2000,6219.19982,2119.29991,0.00000,0.00000,0.00000,2022,June,06-Jun,...,Jun-20,Saturday,MetroBus,Special Service Routes,481-Night Owl North Lamar,10/31/2022,1,2022,9,3
997,19,1524.7739,1524.77390,1524.77390,1372.59995,1372.59995,1372.59995,2022,October,10-Oct,...,Oct-20,Sunday,MetroBus,Local Routes,19-Bull Creek,10/31/2022,1,2023,1,1
998,981,0.0000,0.00000,0.00000,0.00000,0.00000,0.00000,2022,April,04-Apr,...,Apr-20,Weekday,MetroExpress,MetroExpress,981-Oak Knoll Express,10/31/2022,1,2022,7,3
